In [1]:
import numpy as np
from torch import nn,optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

In [2]:
# 训练集
train_dataset = datasets.MNIST(root='./',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
# 测试集
test_dataset = datasets.MNIST(root='./',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=True)

In [3]:
# 批次大小
batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         shuffle=True)

# 装载训练集
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True)

In [4]:
for i,data in enumerate(train_loader):
    inputs,labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1,32,5,1,2),nn.ReLU(),nn.MaxPool2d(2,2))#注意：dropout有时候用效果好有时候用效果不好
        #nn.Conv2d()里面有五个数据，第一个为输入通道数，灰色图片为1，第二个为输出通道数，即输出特征图，第三个参数为卷积窗口大小，第四个为步长大小，第五个参数为padding的0圈数为
        #关于padding有个技巧，当卷积核为3*3时，补充一圈0，当卷积核为5*5时，补充2圈0，当卷积核为3*3时，补充三圈0
        self.conv2 = nn.Sequential(nn.Conv2d(32,64,5,1,2),nn.ReLU(),nn.MaxPool2d(2,2))
        self.fc1 = nn.Sequential(nn.Linear(64*7*7,1000),nn.Dropout(p=0.5),nn.Tanh())#p表示百分之多少的神经元不工作
        self.fc2 = nn.Sequential(nn.Linear(1000,10),nn.Softmax(dim=1))
        
    def forward(self,x):#卷积层只能处理四维的
        
        x = self.conv1(x)
        x = self.conv2(x)
        #全连接层只能处理二维数据,# ->(64,64*7*7)
        x = x.view(x.size()[0],-1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [6]:
LR = 0.001
# 定义模型
model = Net()
# 定义代价函数
mse_loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = optim.Adam(model.parameters(), LR)

In [7]:
def train():
    model.train()#定义模型训练状态，此时dropout起作用
    for i,data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果（64,10）
        out = model(inputs)
        # 交叉熵代价函数out(batch,C), labels(batch)
        loss = mse_loss(out, labels)
        # 梯度清0
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

def test():
    model.eval()#定义模型测试状态，此时dropout不起作用
    correct = 0
    for i,data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(test_dataset)))
    
    
    #训练集的准确率
    correct = 0
    for i,data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Train acc:{0}".format(correct.item()/len(train_dataset)))

In [8]:
for epoch in range(20):
    print('epoch:',epoch)
    train()
    test()

epoch: 0
Test acc:0.9872
Train acc:0.9880833333333333
epoch: 1
Test acc:0.988
Train acc:0.9900833333333333
epoch: 2
Test acc:0.9898
Train acc:0.9928
epoch: 3
Test acc:0.9932
Train acc:0.9951333333333333
epoch: 4
Test acc:0.9925
Train acc:0.99565
epoch: 5
Test acc:0.9906
Train acc:0.9957
epoch: 6
Test acc:0.9916


KeyboardInterrupt: 

In [ ]:
epoch: 0
Test acc:0.8883
epoch: 1
Test acc:0.9008
epoch: 2
Test acc:0.9088
epoch: 3
Test acc:0.9128
epoch: 4
Test acc:0.9138
epoch: 5
Test acc:0.9153
epoch: 6
Test acc:0.916
epoch: 7
Test acc:0.9169
epoch: 8
Test acc:0.9184
epoch: 9
Test acc:0.92